In [0]:
!unzip /content/drive/My\ Drive/dataset_small_class.zip
# ls

In [0]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.image as mpimg
from google.colab.patches import cv2_imshow
from matplotlib import pyplot as plt
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping, ModelCheckpoint

np.random.seed(0)
tf.compat.v1.set_random_seed(3)

In [0]:
imgSize = (48,48)
batchSize = 256
num_classes = 28
epochs = 500

# train_datagen = ImageDataGenerator(validation_split=0.2,
#                                    rescale=1./255)

# train_generator = train_datagen.flow_from_directory(
#         '/content/dataset_small_class',
#         target_size=imgSize,
#         batch_size=batchSize,
#         color_mode='grayscale',
#         class_mode='categorical',
#         subset='training')
# validation_generator = train_datagen.flow_from_directory(
#         '/content/dataset_small_class',
#         target_size=imgSize,    
#         batch_size=batchSize,
#         color_mode='grayscale',
#         class_mode='categorical',
#         subset='validation')
# img = train_generator.filepaths[np.random.random_integers(low=0, high=train_generator.samples)]
# img = mpimg.imread(img)
# cv2_imshow(img)

# x_train, y_train = train_generator.next()
# print(x_train.shape)
# print(validation_generator.class_indices)

# Load Data
path="/content/drive/My Drive/"
x_train, y_train = np.load(path+'train_images.npy'), np.load(path+'train_labels.npy')
x_test,y_test = np.load(path+'test_images.npy'), np.load(path+'test_labels.npy')
mean_tr, std_tr = np.load(path+'mean_tr.npy'), np.load(path+'std_tr.npy')

# Z-Score Normalization
x_train -= mean_tr
x_train /= std_tr
x_test -= mean_tr
x_test /= std_tr

x_train = np.expand_dims(x_train, axis=3)
x_test = np.expand_dims(x_test, axis=3)

# Categorization
y_train = np_utils.to_categorical(y_train)
y_test= np_utils.to_categorical(y_test)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

In [0]:
model = Sequential()
model.add(Conv2D(32, (7, 7), activation='relu', padding='same', input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size=2, strides=2))
model.add(Dropout(0.2))
model.add(Conv2D(64, (7, 7), padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2,strides=2))
model.add(Dropout(0.2))
model.add(Conv2D(128, (7, 7), padding='same',activation='relu'))
model.add(MaxPooling2D(pool_size=2,strides=2))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(num_classes, activation='softmax'))

# model.add(Conv2D(32, (7, 7), activation='relu', input_shape=(48, 48, 1)))
# model.add(Conv2D(32, (7, 7), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Conv2D(64, (7, 7), activation='relu'))
# model.add(Conv2D(64, (7, 7), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))
# model.add(Flatten())
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

In [0]:
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

modelpath="./model/{epoch:02d}-{val_loss:.4f}.h5"

checkpointer = ModelCheckpoint(
    filepath=modelpath, 
    monitor='val_acc', 
    verbose=1, 
    save_best_only=True)

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10)

history = model.fit(x_train, y_train,
                    validation_data=(x_test,y_test),
                    epochs=epochs,
                    callbacks=[early_stopping_callback, checkpointer])

# 모델 평가
# test_datagen = ImageDataGenerator(rescale=1./255)
# test_generator = test_datagen.flow_from_directory(
#         '/content/drive/My Drive/testset_50/',
#         target_size=imgSize,
#         batch_size=batchSize,
#         color_mode='grayscale',
#         class_mode='categorical')
print("-- Evaluate --")
scores = model.evaluate(x_test, y_test)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

# 모델 저장
# model.save('/content/drive/My Drive/simple_v5_gray.h5')

In [0]:
from matplotlib import pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
y_vloss = history.history['val_loss']
y_loss = history.history['loss']

x_len = np.arange(len(y_loss))
plt.plot(x_len, acc, marker='.', c='red',label='Trainset_acc')
plt.plot(x_len, val_acc, marker='.', c='lightcoral',label='Testset_acc')
plt.plot(x_len, y_vloss, marker='.', c='cornflowerblue',label='Testset_loss')
plt.plot(x_len, y_loss, marker='.', c='blue',label='Trainset_loss')

plt.legend(loc='upper right')
plt.grid()
plt.xlabel('epoch')
plt.xlabel('loss/acc')
plt.show

In [0]:
model.save(path+'norm_ext_v2.h5')